# Clustering Crypto

In [32]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [33]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [34]:
#list column names
crypto_df.columns

Index(['Unnamed: 0', 'CoinName', 'Algorithm', 'IsTrading', 'ProofType',
       'TotalCoinsMined', 'TotalCoinSupply'],
      dtype='object')

In [35]:
# Keep all the cryptocurrencies that are being traded.
#new_crypto_df2 = crypto_df.drop(['Unnamed: 0'], axis=1)
new_crypto_trading =crypto_df[crypto_df.IsTrading == True]
new_crypto_trading.count()

Unnamed: 0         1144
CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [36]:
# Keep all the cryptocurrencies that have a working algorithm.
new_crypto_df3 = new_crypto_trading.dropna(subset=['Algorithm'])
new_crypto_df3

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [37]:
# Remove the "IsTrading" column. 
new_crypto_df4 = new_crypto_df3.drop(['IsTrading'],axis=1)
new_crypto_df4.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [38]:
# Remove rows that have at least 1 null value.
new_crypto_df5 = new_crypto_df4.dropna().rename(columns={'Unnamed: 0': ' '})
new_crypto_df5

,,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [39]:
# Keep the rows where coins are mined, set index
new_crypto_df6 = new_crypto_df5[new_crypto_df5.TotalCoinsMined > 0].set_index(' ')
new_crypto_df6.head(5)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [40]:
clean_crypto_df = new_crypto_df6
clean_crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [41]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_names = clean_crypto_df[['CoinName']]
coin_names

,CoinName
,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex


In [42]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_clustering_df = clean_crypto_df.drop(columns='CoinName')
crypto_clustering_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [61]:
# Use get_dummies() to create variables for text features, using SparseArray = True for memory efficiency
X = pd.get_dummies(crypto_clustering_df, sparse= True, columns=['Algorithm', 'ProofType'], drop_first = True)
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
X_scaled

C:\Users\findt\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:515: UserWarning:

pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.

C:\Users\findt\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:515: UserWarning:

pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.



array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [63]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)


In [64]:
# Create a DataFrame with the three principal components.
crypto_pcs_df = pd.DataFrame(data=X_pca, columns=['PC_1', 'PC_2', 'PC_3']).set_index(X.index, drop=False)

crypto_pcs_df.head()

,PC_1,PC_2,PC_3
,,,
42,-0.342809,0.980233,-0.467273
404,-0.326138,0.980434,-0.467819
1337,2.309896,1.692079,-0.594735
BTC,-0.140079,-1.310683,0.167199
ETH,-0.143682,-2.055868,0.271050


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [65]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pcs_df)
    inertia.append(km.inertia_)
    
elbow_data = {'k':k, 'inertia':inertia}
crypto_df_elbow = pd.DataFrame(elbow_data)
crypto_df_elbow.hvplot.line(x='k', y='inertia', xticks=k, title= 'Crypto Elbow Curve')


C:\Users\findt\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [66]:
# Initialize the K-Means model.
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(crypto_pcs_df)

# Predict clusters
predictions = model.predict(crypto_pcs_df)

In [67]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_clustering_df.merge(crypto_pcs_df, left_index=True, right_index=True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coin_names

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.

clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC_1,PC_2,PC_3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.342809,0.980233,-0.467273,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326138,0.980434,-0.467819,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309896,1.692079,-0.594735,EliteCoin,4
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140079,-1.310683,0.167199,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.143682,-2.055868,0.271050,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.173466,-1.176341,0.050394,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.392571,1.305580,-0.551876,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.145477,-2.174972,0.411766,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.142123,-2.055960,0.271029,Ethereum Classic,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [68]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df,
    x="PC_1",
    y='PC_2',
    z='PC_3',
    color='Class',
    symbol='Class',
    width=800,
    hover_name= 'CoinName',
    hover_data=['Algorithm']
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [69]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(
    columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], width=1000, height=600,)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC_1,PC_2,PC_3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.342809,0.980233,-0.467273,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326138,0.980434,-0.467819,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309896,1.692079,-0.594735,EliteCoin,4
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140079,-1.310683,0.167199,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.143682,-2.055868,0.271050,Ethereum,1
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.470107,0.775783,-0.159999,ZEPHYR,4
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.340853,0.980127,-0.467301,Gapcoin,0
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.330454,-2.273651,0.404351,Beldex,1


In [70]:
# Print the total number of tradable cryptocurrencies.
total_tradable_currencies = clustered_df['Class'].count()

print(f'There are {total_tradable_currencies} tradable currencies' )

There are 532 tradable currencies


In [71]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
tradable_cluster = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
scaler = MinMaxScaler().fit_transform(tradable_cluster)
scaler

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [72]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_clustered_df = pd.DataFrame(data = tradable_cluster, index = clustered_df.index)
new_clustered_df["PC 1"] = crypto_pcs_df["PC_1"]
new_clustered_df["PC 2"] = crypto_pcs_df["PC_2"]
new_clustered_df["PC 3"] = crypto_pcs_df["PC_3"]
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
#new_clustered_df["CoinName"] = coin_names["CoinName"] 
new_clustered_df.insert(0, 'CoinName', coin_names)
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
new_clustered_df["Class"] = model.labels_

new_clustered_df.head(10)

,CoinName,TotalCoinSupply,TotalCoinsMined,PC 1,PC 2,PC 3,Class
,,,,,,,
42,42 Coin,42,4.199995e+01,-0.342809,0.980233,-0.467273,0
404,404Coin,532000000,1.055185e+09,-0.326138,0.980434,-0.467819,0
1337,EliteCoin,314159265359,2.927942e+10,2.309896,1.692079,-0.594735,4
BTC,Bitcoin,21000000,1.792718e+07,-0.140079,-1.310683,0.167199,1
ETH,Ethereum,0,1.076842e+08,-0.143682,-2.055868,0.271050,1
LTC,Litecoin,84000000,6.303924e+07,-0.173466,-1.176341,0.050394,1
DASH,Dash,22000000,9.031294e+06,-0.392571,1.305580,-0.551876,0
XMR,Monero,0,1.720114e+07,-0.145477,-2.174972,0.411766,1
ETC,Ethereum Classic,210000000,1.133597e+08,-0.142123,-2.055960,0.271029,1


In [86]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
new_clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    by="Class",
    xlim=(0, 2e+11),
    ylim=(0, 3.5e+11),
    hover_cols=["CoinName"],
    frame_height=500,
    frame_width=500)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

In [77]:
# if we ignore some of the extreme outliers and graph the above by setting lower axis limits
clustered_df.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    xlim=(0, 2e+11),
    ylim=(0, 3.9e+11),
    width=1200, height=600,
    by="Class",
    hover_cols=["CoinName"],
    title = "Total Coins Supplied Vs Mined (AXIS LIMITED!)"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)